<h1 align="center">An Introduction to Machine Learning - 25737</h1>
<h4 align="center">Dr. Sajjad Amini</h4>
<h4 align="center">Sharif University of Technology, Spring 2023</h4>

**Student Name**:

**Student ID**:

# Linear Regression

In this exercise, we want to examine **linear regression**. For this purpose, we have prepared a dataset in the `q1.csv` file. This dataset is used to estimate the **heating load** and **cooling load** of a building based on its parameters. The parameters in this dataset are explained below:

- $X_1$: Relative Compactness
- $X_2$: Surface Area
- $X_3$: Wall Area
- $X_4$: Roof Area
- $X_5$: Overall Height
- $X_6$: Orientation
- $X_7$: Glazing Area
- $X_8$: Glazing Area Distribution
- $Y_1$: Heating Load
- $Y_2$: Cooling Load

**Note**: For the sake of simplicity, we will only focus on estimating the **heating load** in this problem. Also, please note that we have some inline questions in this notebook, for which you should write your answers in the **Answer** section below each question.

## Importing Libraries

First we import libraries that we need for this assignment.

**Attention**: You should only use these libraries. Other libraries are not acceptable.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Reading Data and Preprocessing

In this section, we want to read data from a CSV file and then preprocess it to make it ready for the rest of the problem.

First, we read the data in the cell below and extract an $m \times n$ matrix, $X$, and an $m \times 1$ vector, $Y$, from it, which represent our knowledge about the building (`X1`, `X2`, ..., `X8`) and heating load (`Y1`), respectively. Note that by $m$, we mean the number of data points and by $n$, we mean the number of features.

In [2]:
X, Y = None, None

### START CODE HERE ###
data_path = "q1.csv"
df = pd.read_csv(data_path)
X = df.iloc[0:,0:8].to_numpy()
Y = df['Y1'].to_numpy()
### END CODE HERE ###

print(X.shape)
print(Y.shape)

(768, 8)
(768,)


Next, we should normalize our data. For normalizing a vector $\mathbf{x}$, a very common method is to use this formula:

$$
\mathbf{x}_{norm} = \dfrac{\mathbf{x} - \overline{\mathbf{x}}}{\sigma_\mathbf{x}}
$$

Here, $\overline{x}$ and $\sigma_\mathbf{x}$ denote the mean and standard deviation of vector $\mathbf{x}$, respectively. Use this formula and store the new $X$ and $Y$ vectors in the cell below.

**Question**: Briefly explain why we need to normalize our data before starting the training.

**Answer**: By normalizing, convergence would be easier and our training will be less sesitive to the scale.

In [3]:
### START CODE HERE ###
m,n = X.shape
X_mean = np.mean(X, axis=0)
#print(X_mean)
print(np.linalg.norm(X-X_mean, axis=0)/np.sqrt(m))
X = (X-X_mean) / (np.linalg.norm(X-X_mean, axis=0)/np.sqrt(m))

Y_mean = np.mean(Y,axis=0)
Y = (Y-Y_mean)/(np.linalg.norm(Y-Y_mean)/np.sqrt(m))
### END CODE HERE ###

[ 0.10570859 88.02874964 43.59806953 45.13653573  1.75        1.11803399
  0.1331338   1.5499496 ]


Finally, we should add a column of $1$s at the beginning of $X$ to represent the bias term. Do this in the next cell. Note that after this process, $X$ should be an $m \times (n+1)$ matrix.

In [4]:
### START CODE HERE ###
n, m = X.shape
X_bias = np.hstack((np.ones((n, 1)), X))
norms = np.linalg.norm(X_bias, axis=1, keepdims=True)
X = X_bias / norms
### END CODE HERE ###

print(X.shape)

(768, 9)


## Training Model Using Direct Method

We know that the loss function in linear regression is defined as:

$$
\mathcal{L}(\mathbf{w}) = \frac{1}{m}\sum_{i=1}^{m}(\mathbf{w}^\top\mathbf{x}_i-y_i)^2
$$

Here, $w$ is the weight vector and $(x_i, y_i)$ represents the $i$th data point. First, write a function that takes $X$, $Y$, and $w$ as inputs and returns the loss value in the next cell. Note that your implementation should be fully vectorized, meaning that you are not allowed to use any loops in your function and should only use functions prepared in the numpy library.

In [5]:
def loss(X, Y, w):
  '''
  X: an m by (n+1) matrix which includes inputs
  Y: an m by 1 vector which includes heating loads
  w: an (n+1) by 1 weight vector
  '''
  m, n = X.shape
  
  ### START CODE HERE ###
  loss = 1/m*(np.linalg.norm(X @ w - Y))**2
  ### END CODE HERE ###
  return loss

Now, we want to calculate the weight matrix, $w$, using the direct method. By direct method, we mean finding the answer to the optimization problem below directly using linear algebra, without using iterative methods:

$$
\min_{w} \mathcal{L}(w)
$$

Question: What is the answer to this problem in terms of $X$ and $Y$?

Answer:

It is a minimum square problem. we reformulate it:
$$
\min_{w} ||X w - Y||^2
$$
we solve it by :
$
w^* = (X^T X)^{-1}X^T Y
$

Now you should implement a function that receives $X$ and $Y$ as input and returns $w$. Note that your implementation should also be fully vectorized.

In [6]:
def direct_method(X, Y):
  '''
  X: an m by (n+1) matrix which includes inputs
  Y: an m by 1 vector which includes heating loads
  '''
  w = None
  ### START CODE HERE ###
  w = (np.linalg.pinv((X.T @ X))@X.T@Y)
  ### END CODE HERE ###
  return w

Finally, we want to evaluate our loss for this problem. Run the cell below to calculate the loss of your model.

In [7]:
w = direct_method(X, Y) # calculating w using direct method
print(w)
print(f"loss for this problem using direct method is {loss(X, Y, w)}")

[ 0.03505873 -3.09186771 -2.44069238  0.33565278 -2.54211924  0.46268615
 -0.00734704  0.75464899  0.10021754]
loss for this problem using direct method is 0.08840856907902836


## Training Model Using Gradient Descent

Now, instead of using the direct method to calculate $w$, we want to use the **Gradient Descent** algorithm. We know that in this algorithm, in each iteration, we should update our weight vector with:

$$
\mathbf{w}^{(t+1)} = \mathbf{w}^{(t)} - \alpha \nabla \mathcal{L}(\mathbf{w}^{(t)})
$$

Here, $w^{t}$ represents the weight matrix in the $t$th iteration, and $\alpha$ represents the learning rate.

**Question**: Write an expression for $\nabla\mathcal{L}(\mathbf{w})$.

**Answer**:

$$
\nabla ||X w - Y||^2 = \nabla (X w - Y)^T(X w - Y) = \nabla (w^TX^TXw + Y^TY + 2Y^TXw) 
$$
by vecctor differentiation identities we have:
$$
\nabla (w^T X^T X w) = (X^T X + (X^T X)^T) w = 2 X^T X w
$$

$$
\nabla  (2Y^T X w) = 2 Y^T X
$$
so we have
$$
\nabla L(w) = \frac{1}{m}(2 X^T X w - 2 X^T Y)
$$
Now, write a function that computes the gradient of $\mathcal{L}(\mathbf{w})$. This function should receive $X$, $Y$, and $\mathbf{w}$ as inputs and return an $(n+1) \times 1$ vector, which represents $\nabla\mathcal{L}(\mathbf{w})$. Note that your implementation should also be **fully vectorized**.

In [8]:
def gradient(X, Y, w):
  '''
  X: an m by (n+1) matrix which includes inputs
  Y: an m by 1 vector which includes heating loads
  w: an (n+1) by 1 weight vector
  '''
  m, n = X.shape
  grad = None
  ### START CODE HERE ###
  
  grad = (2 * X.T @ X @ w - 2 * (X.T@Y))/m
  ### END CODE HERE ###
  return grad

Now, we are ready to implement the Gradient Descent algorithm. Complete the function below for this purpose. Note that this function receives $X$, $Y$, the learning rate, and the number of iterations as inputs. This function should return two parameters. The first parameter is $\mathbf{w}$, and the second parameter is a `numpy` array that contains the loss in each iteration. This array is indicated by `loss_history` in the code. Also note that you should initialize $\mathbf{w}$ with the `randn` function.

In [9]:
def gradient_descent(X, Y, alpha, num_iter):
  '''
  X: an m by (n+1) matrix which includes inputs
  Y: an m by 1 vector which includes heating loads
  alpha: learning rate
  num_iter: number of iterations of the algorithm
  '''
  m, n = X.shape
  w, loss_history = None,None
  ### START CODE HERE ###
  w = np.random.randn(n)

  loss_history = []
  for i in range(num_iter):
    w = w - alpha * gradient(X,Y,w)
    

    loss_history.append(loss(X,Y,w))
    
  ### END CODE HERE ###
  return w, loss_history

Now, run the `gradient_descent` function for 5 different values of the learning rate. Plot the `loss_history` of these 5 different values in the same figure.

**Question**: Discuss the effect of the learning rate and find the best value of this parameter.

**Answer**:

In [10]:
### START CODE HERE ###
n = 100
alpha = 0.1
w,loss_history = gradient_descent(X,Y,alpha,n)
print("loss with alpha = {} :{}".format(alpha,loss_history[n-1]))

alpha = 0.5
w,loss_history = gradient_descent(X,Y,alpha,n)
print("loss with alpha = {} :{}".format(alpha,loss_history[n-1]))

alpha = 1
w,loss_history = gradient_descent(X,Y,alpha,n)
print("loss with alpha = {} :{}".format(alpha,loss_history[n-1]))

alpha = 1.5
w,loss_history = gradient_descent(X,Y,alpha,n)
print("loss with alpha = {} :{}".format(alpha,loss_history[n-1]))

alpha = 2
w,loss_history = gradient_descent(X,Y,alpha,n)
print("loss with alpha = {} :{}".format(alpha,loss_history[n-1]))

### END CODE HERE ###

loss with alpha = 0.1 :0.12951242086136122
loss with alpha = 0.5 :0.11571532125110912
loss with alpha = 1 :0.095414757285621
loss with alpha = 1.5 :0.09725452388857027
loss with alpha = 2 :0.09203067375826407


## Conclusion

Compare the answer of two different methods that we used earlier.

**Question**: Discuss these two methods and compare them with each other. When is it better to use the direct method, and when is it better to use Gradient Descent?

**Answer**: direct method can be used when we can minimize loss function anylytically.but in some cases we can not therefore we use numerical methods like gradient descent. The problem with gradient descent is that learning rate is so important. if it is chosen very high it will not converge and if it is chosen very low it will converge too late.

## (Additional Part) Stochastic Gradient Descent

When the number of data points becomes large, calculating the gradient becomes very complicated. In these circumstances, we use **Stochastic Gradient Descent**. In this algorithm, instead of using all of the data points to calculate the gradient, we use only a small number of them. We choose these small number of points randomly in each iteration. Implement this algorithm, and use it to calculate $w$, and then compare the result with the preceding parts.

In [11]:
def stochastic_gradient_descent(X, Y, k, alpha, num_iter):
  '''
  X: an m by (n+1) matrix which includes inputs
  Y: an m by 1 vector which includes heating loads
  k: number of data points used in each iteration
  alpha: learning rate
  num_iter: number of iterations of the algorithm
  '''
  m, n = X.shape
  w, loss_history = None, None 
  ### START CODE HERE ###
  w = np.random.randn(n)

  loss_history = []
  for i in range(num_iter):
    indexes = np.random.randint(m, size=k)
    w = w - alpha * gradient(X[indexes,:],Y[indexes],w)
    

    loss_history.append(loss(X,Y,w))
    
  ### END CODE HERE ###
  return w, loss_history

In [12]:
n = 100
k = 50
alpha = 0.1
w,loss_history = stochastic_gradient_descent(X, Y, k, alpha, n)
print("loss with alpha = {} :{}".format(alpha,loss_history[n-1]))

alpha = 0.5
w,loss_history = stochastic_gradient_descent(X, Y, k, alpha, n)
print("loss with alpha = {} :{}".format(alpha,loss_history[n-1]))

alpha = 1
w,loss_history = stochastic_gradient_descent(X, Y, k, alpha, n)
print("loss with alpha = {} :{}".format(alpha,loss_history[n-1]))

alpha = 1.5
w,loss_history = stochastic_gradient_descent(X, Y, k, alpha, n)
print("loss with alpha = {} :{}".format(alpha,loss_history[n-1]))

alpha = 2
w,loss_history = stochastic_gradient_descent(X, Y, k, alpha, n)
print("loss with alpha = {} :{}".format(alpha,loss_history[n-1]))


loss with alpha = 0.1 :0.11581950028043832
loss with alpha = 0.5 :0.09944986922776324
loss with alpha = 1 :0.09733119397687037
loss with alpha = 1.5 :0.10289125154533343
loss with alpha = 2 :0.10033549525069704


It is shown that the performance of normal gradient descent is slightly better however stochastic is faster.